In [52]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn_pandas import DataFrameMapper, cross_val_score
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_extraction import WordCount, CharCount, DataExtractor, ItemSelector, Vowels, Consonants, Polarity, Subjectivity, Nouns, Blob, Words
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import TruncatedSVD

In [58]:
#data = pd.read_csv('processed_posts.csv.bz2')
data = pd.read_csv('processed_datascience.csv')
data.drop_duplicates('title', inplace=True)

data['gt20'] = data['ups'] > 20
# data['wordcount'] = data['title'].str.split().apply(len)
# data['charcount'] = data.title.str.len()
# data['vowels'] = data.title.str.findall(r'[aeiou]').apply(len)
# data['consonants'] = data.title.str.findall(r'[^aeiou]').apply(len)

#data = data[data['subreddit'] == 'r/books']
train_X, test_X, train_y, test_y = train_test_split(data.title, 
                                                    data.gt20, 
                                                    test_size=0.20,
                                                    random_state=42)

In [59]:
print(data[data['gt20']==False].shape)
print(data[data['gt20']==True].shape)


(1685, 7)
(312, 7)


In [67]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('words', Words()),
                      
            ('title', Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(1,3), sublinear_tf=True, stop_words='english')),
                ('svd', TruncatedSVD(n_components=120)),
                ('normalize', MinMaxScaler(copy=False)),
                ('selector', SelectPercentile(f_classif, percentile=10))
            ])),
            
            ('blob', Pipeline([
                ('all', Blob()),
                ('minmax', MinMaxScaler()),
            ])),
            
            ])),
    ('clf', ExtraTreesClassifier()),
        ])

In [68]:
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('words', Words()), ('title', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...timators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [69]:
y = pipeline.predict(test_X)
accuracy_score(y_pred=y, y_true=test_y)

0.82499999999999996

In [63]:
cross_val_score(pipeline, train_X, train_y, cv=5)

array([ 0.846875  ,  0.83125   ,  0.821875  ,  0.8369906 ,  0.83647799])

In [72]:
print (test_X.iloc[1])
print (test_y.iloc[1])
print (y[1])
pipeline.predict(pd.Series(['How I went from no coding or machine learning experience to data scientist job offer in 20 months']))

Starting dataset for learning recommender systems?
False
False


array([False], dtype=bool)

In [ ]:
data.to_csv('gt_politics.csv', index=False)

In [ ]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('wordcount', WordCount()),
            
            ('charcount', CharCount()),
            
            ('title', TfidfVectorizer(ngram_range=(1,4))),
            
            ('vowel', Vowels()),
            
            ('consonants', Consonants()),
            
            ('polarity', Polarity()),
            
            ('subjectivity', Subjectivity()),
            
            ('noun_phrases', Nouns()),
            
            ])),
    ('clf', ExtraTreesClassifier()),
        ])